# Ranking

## Load Data

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [3]:
cat_cols = [x for x in train.columns if x.startswith('cat')]
cat_cols

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18']

In [4]:
x_train = train.drop(columns=['id','target'])
y_train = train['target']

In [5]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)

## Default Lightgbm Model

In [7]:
from categorical_transform import CategoricalTransform
from lightgbm.sklearn import LGBMRanker
from sklearn.pipeline import Pipeline

In [8]:
lightgbm_pipe = Pipeline([("trans", CategoricalTransform(cat_cols)),
                          ("lgbm", LGBMRanker(n_jobs = -2, objective='rank_xendcg'))])

In [25]:
lightgbm_pipe.fit(x_train, y_train, lgbm__group=[len(x_train)])

Pipeline(steps=[('trans',
                 CategoricalTransform(cat_cols=['cat0', 'cat1', 'cat2', 'cat3',
                                                'cat4', 'cat5', 'cat6', 'cat7',
                                                'cat8', 'cat9', 'cat10',
                                                'cat11', 'cat12', 'cat13',
                                                'cat14', 'cat15', 'cat16',
                                                'cat17', 'cat18'])),
                ('lgbm', LGBMRanker(n_jobs=-2, objective='rank_xendcg'))])

In [26]:
pred = lightgbm_pipe.predict(x_train)
print(f"min(pred)={min(pred)}, max(pred)={max(pred)}")

min(pred)=-0.5325426467476356, max(pred)=0.9253941786924191


In [7]:
import numpy as np
from sklearn.metrics import roc_auc_score

def normalize_ranks(rank_values):
    rank_range = max(rank_values) - min(rank_values)
    return (rank_values-min(rank_values))/rank_range

def cv_train(pipe, fit_params):
    kf = KFold(n_splits=5, shuffle=True)
    roc_test = []
    roc_train = []
    for train_index, test_index in kf.split(train):
        train_folds = train.loc[train_index]
        test_fold = train.loc[test_index]
        x_train = train_folds.drop(columns=['id','target'])
        y_train = train_folds['target'] 
        pipe.fit(x_train, y_train, **fit_params)

        x_test = test_fold.drop(columns=['id','target'])
        y_test = test_fold['target'] 
        rank_value = normalize_ranks(pipe.predict(x_test))
        roc_test.append(roc_auc_score(y_test, rank_value))

        rank_value = normalize_ranks(pipe.predict(x_train))
        roc_train.append(roc_auc_score(y_train, rank_value))
    return roc_test, roc_train

In [28]:
roc_test, roc_train = cv_train(lightgbm_pipe, fit_params = dict(lgbm__group=[4/5*len(train)]))

In [29]:
roc_test

[0.884550569099091,
 0.8852800920286608,
 0.8838539719725717,
 0.8862357836528423,
 0.8867424068556633]

In [30]:
import numpy as np
np.mean(roc_test)

0.8853325647217659

In [31]:
np.mean(roc_train)

0.8912101641097067

## Default CatBoost

In [8]:
from categorical_transform import IntegerCategoricalTransform
from catboost import CatBoost
from sklearn.pipeline import Pipeline

In [42]:
cbc = CatBoost(params={'cat_features':cat_cols, 'thread_count':6, 'objective':'PairLogit:max_pairs=400000'})
catboost_pipe = Pipeline([("trans", IntegerCategoricalTransform(cat_cols)),
                          ("catboost", cbc)])

In [43]:
catboost_pipe.fit(x_train, y_train, catboost__group_id=[1]*len(x_train))

Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.6635727	total: 1.36s	remaining: 22m 41s
1:	learn: 0.6358060	total: 2.68s	remaining: 22m 16s
2:	learn: 0.6103427	total: 3.97s	remaining: 21m 58s
3:	learn: 0.5870655	total: 5.22s	remaining: 21m 40s
4:	learn: 0.5663600	total: 6.74s	remaining: 22m 21s
5:	learn: 0.5473521	total: 8.03s	remaining: 22m 9s
6:	learn: 0.5291619	total: 9.32s	remaining: 22m 2s
7:	learn: 0.5124067	total: 10.6s	remaining: 21m 57s
8:	learn: 0.4975984	total: 11.9s	remaining: 21m 53s
9:	learn: 0.4839536	total: 13.2s	remaining: 21m 48s
10:	learn: 0.4712259	total: 14.6s	remaining: 21m 49s
11:	learn: 0.4597427	total: 15.8s	remaining: 21m 41s
12:	learn: 0.4501862	total: 17s	remaining: 21m 30s
13:	learn: 0.4407166	total: 18.5s	remaining: 21m 42s
14:	learn: 0.4314676	total: 19.7s	remaining: 21m 35s
15:	learn: 0.4236611	total: 21s	remaining: 21m 33s
16:	learn: 0.4160164	total: 22.4s	remaining: 21m 35s
17:	learn: 0.4094710	total: 23.7s	remaining: 21m 34s
18:	learn: 0.

153:	learn: 0.2826937	total: 3m 19s	remaining: 18m 17s
154:	learn: 0.2825826	total: 3m 21s	remaining: 18m 16s
155:	learn: 0.2824066	total: 3m 22s	remaining: 18m 15s
156:	learn: 0.2822591	total: 3m 23s	remaining: 18m 13s
157:	learn: 0.2821089	total: 3m 24s	remaining: 18m 12s
158:	learn: 0.2819717	total: 3m 26s	remaining: 18m 11s
159:	learn: 0.2818642	total: 3m 27s	remaining: 18m 9s
160:	learn: 0.2817211	total: 3m 28s	remaining: 18m 8s
161:	learn: 0.2816119	total: 3m 30s	remaining: 18m 7s
162:	learn: 0.2814722	total: 3m 31s	remaining: 18m 5s
163:	learn: 0.2813782	total: 3m 32s	remaining: 18m 4s
164:	learn: 0.2812641	total: 3m 33s	remaining: 18m 2s
165:	learn: 0.2811787	total: 3m 35s	remaining: 18m
166:	learn: 0.2810598	total: 3m 36s	remaining: 17m 59s
167:	learn: 0.2809747	total: 3m 37s	remaining: 17m 57s
168:	learn: 0.2808492	total: 3m 38s	remaining: 17m 56s
169:	learn: 0.2807201	total: 3m 40s	remaining: 17m 54s
170:	learn: 0.2805519	total: 3m 41s	remaining: 17m 52s
171:	learn: 0.280452

304:	learn: 0.2697622	total: 6m 35s	remaining: 15m
305:	learn: 0.2697068	total: 6m 36s	remaining: 14m 59s
306:	learn: 0.2696685	total: 6m 37s	remaining: 14m 58s
307:	learn: 0.2696217	total: 6m 39s	remaining: 14m 56s
308:	learn: 0.2695414	total: 6m 40s	remaining: 14m 55s
309:	learn: 0.2695010	total: 6m 41s	remaining: 14m 54s
310:	learn: 0.2694555	total: 6m 42s	remaining: 14m 52s
311:	learn: 0.2694096	total: 6m 44s	remaining: 14m 51s
312:	learn: 0.2693753	total: 6m 45s	remaining: 14m 50s
313:	learn: 0.2693213	total: 6m 47s	remaining: 14m 49s
314:	learn: 0.2692775	total: 6m 48s	remaining: 14m 48s
315:	learn: 0.2692029	total: 6m 49s	remaining: 14m 46s
316:	learn: 0.2691321	total: 6m 50s	remaining: 14m 45s
317:	learn: 0.2691033	total: 6m 52s	remaining: 14m 44s
318:	learn: 0.2690509	total: 6m 53s	remaining: 14m 42s
319:	learn: 0.2689695	total: 6m 54s	remaining: 14m 41s
320:	learn: 0.2689051	total: 6m 56s	remaining: 14m 40s
321:	learn: 0.2688670	total: 6m 57s	remaining: 14m 39s
322:	learn: 0.

455:	learn: 0.2621702	total: 9m 53s	remaining: 11m 48s
456:	learn: 0.2621196	total: 9m 55s	remaining: 11m 47s
457:	learn: 0.2620584	total: 9m 56s	remaining: 11m 45s
458:	learn: 0.2620288	total: 9m 57s	remaining: 11m 44s
459:	learn: 0.2619895	total: 9m 59s	remaining: 11m 43s
460:	learn: 0.2619336	total: 10m	remaining: 11m 41s
461:	learn: 0.2618664	total: 10m 1s	remaining: 11m 40s
462:	learn: 0.2618107	total: 10m 2s	remaining: 11m 39s
463:	learn: 0.2617719	total: 10m 4s	remaining: 11m 37s
464:	learn: 0.2617217	total: 10m 5s	remaining: 11m 36s
465:	learn: 0.2616753	total: 10m 6s	remaining: 11m 35s
466:	learn: 0.2616310	total: 10m 7s	remaining: 11m 33s
467:	learn: 0.2615937	total: 10m 9s	remaining: 11m 32s
468:	learn: 0.2615533	total: 10m 10s	remaining: 11m 31s
469:	learn: 0.2615121	total: 10m 12s	remaining: 11m 30s
470:	learn: 0.2614456	total: 10m 13s	remaining: 11m 28s
471:	learn: 0.2613989	total: 10m 14s	remaining: 11m 27s
472:	learn: 0.2613427	total: 10m 16s	remaining: 11m 26s
473:	lea

604:	learn: 0.2555217	total: 13m 10s	remaining: 8m 36s
605:	learn: 0.2554793	total: 13m 12s	remaining: 8m 34s
606:	learn: 0.2554396	total: 13m 13s	remaining: 8m 33s
607:	learn: 0.2554017	total: 13m 14s	remaining: 8m 32s
608:	learn: 0.2553588	total: 13m 15s	remaining: 8m 31s
609:	learn: 0.2553071	total: 13m 17s	remaining: 8m 29s
610:	learn: 0.2552773	total: 13m 18s	remaining: 8m 28s
611:	learn: 0.2552370	total: 13m 19s	remaining: 8m 27s
612:	learn: 0.2551981	total: 13m 21s	remaining: 8m 25s
613:	learn: 0.2551504	total: 13m 22s	remaining: 8m 24s
614:	learn: 0.2551116	total: 13m 24s	remaining: 8m 23s
615:	learn: 0.2550640	total: 13m 25s	remaining: 8m 22s
616:	learn: 0.2550379	total: 13m 27s	remaining: 8m 20s
617:	learn: 0.2549929	total: 13m 28s	remaining: 8m 19s
618:	learn: 0.2549500	total: 13m 29s	remaining: 8m 18s
619:	learn: 0.2549033	total: 13m 31s	remaining: 8m 17s
620:	learn: 0.2548706	total: 13m 32s	remaining: 8m 15s
621:	learn: 0.2548280	total: 13m 33s	remaining: 8m 14s
622:	learn

755:	learn: 0.2500155	total: 16m 33s	remaining: 5m 20s
756:	learn: 0.2499842	total: 16m 35s	remaining: 5m 19s
757:	learn: 0.2499591	total: 16m 36s	remaining: 5m 18s
758:	learn: 0.2499243	total: 16m 38s	remaining: 5m 16s
759:	learn: 0.2498969	total: 16m 39s	remaining: 5m 15s
760:	learn: 0.2498560	total: 16m 40s	remaining: 5m 14s
761:	learn: 0.2498124	total: 16m 42s	remaining: 5m 12s
762:	learn: 0.2497856	total: 16m 43s	remaining: 5m 11s
763:	learn: 0.2497449	total: 16m 44s	remaining: 5m 10s
764:	learn: 0.2497225	total: 16m 45s	remaining: 5m 9s
765:	learn: 0.2496906	total: 16m 47s	remaining: 5m 7s
766:	learn: 0.2496561	total: 16m 48s	remaining: 5m 6s
767:	learn: 0.2496106	total: 16m 50s	remaining: 5m 5s
768:	learn: 0.2495801	total: 16m 51s	remaining: 5m 3s
769:	learn: 0.2495580	total: 16m 52s	remaining: 5m 2s
770:	learn: 0.2495187	total: 16m 54s	remaining: 5m 1s
771:	learn: 0.2494801	total: 16m 55s	remaining: 4m 59s
772:	learn: 0.2494380	total: 16m 57s	remaining: 4m 58s
773:	learn: 0.249

906:	learn: 0.2452949	total: 19m 59s	remaining: 2m 2s
907:	learn: 0.2452640	total: 20m	remaining: 2m 1s
908:	learn: 0.2452401	total: 20m 1s	remaining: 2m
909:	learn: 0.2452124	total: 20m 3s	remaining: 1m 59s
910:	learn: 0.2451834	total: 20m 4s	remaining: 1m 57s
911:	learn: 0.2451548	total: 20m 6s	remaining: 1m 56s
912:	learn: 0.2451175	total: 20m 7s	remaining: 1m 55s
913:	learn: 0.2450886	total: 20m 8s	remaining: 1m 53s
914:	learn: 0.2450575	total: 20m 10s	remaining: 1m 52s
915:	learn: 0.2450234	total: 20m 11s	remaining: 1m 51s
916:	learn: 0.2449962	total: 20m 12s	remaining: 1m 49s
917:	learn: 0.2449682	total: 20m 14s	remaining: 1m 48s
918:	learn: 0.2449423	total: 20m 15s	remaining: 1m 47s
919:	learn: 0.2449086	total: 20m 16s	remaining: 1m 45s
920:	learn: 0.2448817	total: 20m 18s	remaining: 1m 44s
921:	learn: 0.2448447	total: 20m 19s	remaining: 1m 43s
922:	learn: 0.2447955	total: 20m 21s	remaining: 1m 41s
923:	learn: 0.2447747	total: 20m 22s	remaining: 1m 40s
924:	learn: 0.2447492	tota

Pipeline(steps=[('trans',
                 IntegerCategoricalTransform(cat_cols=['cat0', 'cat1', 'cat2',
                                                       'cat3', 'cat4', 'cat5',
                                                       'cat6', 'cat7', 'cat8',
                                                       'cat9', 'cat10', 'cat11',
                                                       'cat12', 'cat13',
                                                       'cat14', 'cat15',
                                                       'cat16', 'cat17',
                                                       'cat18'])),
                ('catboost',
                 <catboost.core.CatBoost object at 0x7f9310c63a30>)])

In [44]:
roc_test, roc_train = cv_train(catboost_pipe, dict(catboost__group_id=[1]*int(4/5*len(x_train))))

Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.6643257	total: 1.27s	remaining: 21m 9s
1:	learn: 0.6386006	total: 2.33s	remaining: 19m 22s
2:	learn: 0.6132346	total: 3.51s	remaining: 19m 27s
3:	learn: 0.5909248	total: 4.63s	remaining: 19m 13s
4:	learn: 0.5692096	total: 5.68s	remaining: 18m 49s
5:	learn: 0.5493623	total: 6.73s	remaining: 18m 34s
6:	learn: 0.5317534	total: 8.17s	remaining: 19m 19s
7:	learn: 0.5151572	total: 9.21s	remaining: 19m 1s
8:	learn: 0.5008468	total: 10.4s	remaining: 19m 2s
9:	learn: 0.4866212	total: 11.4s	remaining: 18m 50s
10:	learn: 0.4739115	total: 12.5s	remaining: 18m 42s
11:	learn: 0.4617888	total: 13.6s	remaining: 18m 35s
12:	learn: 0.4516471	total: 14.6s	remaining: 18m 25s
13:	learn: 0.4420653	total: 15.6s	remaining: 18m 18s
14:	learn: 0.4335118	total: 16.7s	remaining: 18m 15s
15:	learn: 0.4256414	total: 18.1s	remaining: 18m 31s
16:	learn: 0.4179902	total: 19.3s	remaining: 18m 33s
17:	learn: 0.4105686	total: 20.3s	remaining: 18m 28s
18:	learn:

153:	learn: 0.2845495	total: 2m 52s	remaining: 15m 46s
154:	learn: 0.2844446	total: 2m 53s	remaining: 15m 44s
155:	learn: 0.2843310	total: 2m 54s	remaining: 15m 43s
156:	learn: 0.2841729	total: 2m 55s	remaining: 15m 42s
157:	learn: 0.2840423	total: 2m 56s	remaining: 15m 40s
158:	learn: 0.2838863	total: 2m 57s	remaining: 15m 39s
159:	learn: 0.2838090	total: 2m 58s	remaining: 15m 38s
160:	learn: 0.2837166	total: 2m 59s	remaining: 15m 37s
161:	learn: 0.2836294	total: 3m 1s	remaining: 15m 36s
162:	learn: 0.2834626	total: 3m 2s	remaining: 15m 35s
163:	learn: 0.2833719	total: 3m 3s	remaining: 15m 34s
164:	learn: 0.2832646	total: 3m 4s	remaining: 15m 32s
165:	learn: 0.2831625	total: 3m 5s	remaining: 15m 31s
166:	learn: 0.2830803	total: 3m 6s	remaining: 15m 30s
167:	learn: 0.2829925	total: 3m 7s	remaining: 15m 29s
168:	learn: 0.2828930	total: 3m 8s	remaining: 15m 27s
169:	learn: 0.2827879	total: 3m 9s	remaining: 15m 27s
170:	learn: 0.2826653	total: 3m 11s	remaining: 15m 26s
171:	learn: 0.28257

304:	learn: 0.2721936	total: 5m 42s	remaining: 13m
305:	learn: 0.2721362	total: 5m 43s	remaining: 12m 59s
306:	learn: 0.2720819	total: 5m 44s	remaining: 12m 58s
307:	learn: 0.2720348	total: 5m 45s	remaining: 12m 57s
308:	learn: 0.2719707	total: 5m 46s	remaining: 12m 55s
309:	learn: 0.2719081	total: 5m 48s	remaining: 12m 54s
310:	learn: 0.2718334	total: 5m 49s	remaining: 12m 53s
311:	learn: 0.2717901	total: 5m 50s	remaining: 12m 52s
312:	learn: 0.2717448	total: 5m 51s	remaining: 12m 51s
313:	learn: 0.2716726	total: 5m 52s	remaining: 12m 50s
314:	learn: 0.2716057	total: 5m 53s	remaining: 12m 49s
315:	learn: 0.2715499	total: 5m 54s	remaining: 12m 48s
316:	learn: 0.2714953	total: 5m 55s	remaining: 12m 46s
317:	learn: 0.2714404	total: 5m 57s	remaining: 12m 45s
318:	learn: 0.2714018	total: 5m 58s	remaining: 12m 44s
319:	learn: 0.2713553	total: 5m 59s	remaining: 12m 43s
320:	learn: 0.2713094	total: 6m	remaining: 12m 42s
321:	learn: 0.2712667	total: 6m 1s	remaining: 12m 41s
322:	learn: 0.27121

455:	learn: 0.2642563	total: 8m 32s	remaining: 10m 11s
456:	learn: 0.2642095	total: 8m 33s	remaining: 10m 9s
457:	learn: 0.2641672	total: 8m 34s	remaining: 10m 8s
458:	learn: 0.2641235	total: 8m 35s	remaining: 10m 7s
459:	learn: 0.2640721	total: 8m 36s	remaining: 10m 6s
460:	learn: 0.2640175	total: 8m 38s	remaining: 10m 5s
461:	learn: 0.2639468	total: 8m 39s	remaining: 10m 4s
462:	learn: 0.2638578	total: 8m 40s	remaining: 10m 3s
463:	learn: 0.2638273	total: 8m 41s	remaining: 10m 2s
464:	learn: 0.2637865	total: 8m 42s	remaining: 10m 1s
465:	learn: 0.2637214	total: 8m 43s	remaining: 9m 59s
466:	learn: 0.2636721	total: 8m 44s	remaining: 9m 58s
467:	learn: 0.2636147	total: 8m 45s	remaining: 9m 57s
468:	learn: 0.2635595	total: 8m 46s	remaining: 9m 56s
469:	learn: 0.2635028	total: 8m 48s	remaining: 9m 55s
470:	learn: 0.2634302	total: 8m 49s	remaining: 9m 54s
471:	learn: 0.2633912	total: 8m 50s	remaining: 9m 53s
472:	learn: 0.2633405	total: 8m 51s	remaining: 9m 52s
473:	learn: 0.2632819	total

607:	learn: 0.2571123	total: 11m 25s	remaining: 7m 21s
608:	learn: 0.2570752	total: 11m 26s	remaining: 7m 20s
609:	learn: 0.2570333	total: 11m 27s	remaining: 7m 19s
610:	learn: 0.2569945	total: 11m 28s	remaining: 7m 18s
611:	learn: 0.2569582	total: 11m 29s	remaining: 7m 17s
612:	learn: 0.2569286	total: 11m 30s	remaining: 7m 16s
613:	learn: 0.2568989	total: 11m 31s	remaining: 7m 14s
614:	learn: 0.2568545	total: 11m 32s	remaining: 7m 13s
615:	learn: 0.2568116	total: 11m 33s	remaining: 7m 12s
616:	learn: 0.2567681	total: 11m 35s	remaining: 7m 11s
617:	learn: 0.2567234	total: 11m 36s	remaining: 7m 10s
618:	learn: 0.2566921	total: 11m 37s	remaining: 7m 9s
619:	learn: 0.2566433	total: 11m 38s	remaining: 7m 8s
620:	learn: 0.2565884	total: 11m 39s	remaining: 7m 7s
621:	learn: 0.2565543	total: 11m 40s	remaining: 7m 5s
622:	learn: 0.2565113	total: 11m 42s	remaining: 7m 4s
623:	learn: 0.2564743	total: 11m 43s	remaining: 7m 3s
624:	learn: 0.2564294	total: 11m 44s	remaining: 7m 2s
625:	learn: 0.256

...

391:	learn: 0.2653412	total: 7m 33s	remaining: 11m 42s
392:	learn: 0.2652986	total: 7m 34s	remaining: 11m 41s
393:	learn: 0.2652262	total: 7m 35s	remaining: 11m 40s
394:	learn: 0.2651723	total: 7m 36s	remaining: 11m 38s
395:	learn: 0.2650883	total: 7m 37s	remaining: 11m 37s
396:	learn: 0.2650332	total: 7m 38s	remaining: 11m 36s
397:	learn: 0.2649757	total: 7m 39s	remaining: 11m 35s
398:	learn: 0.2648867	total: 7m 40s	remaining: 11m 34s
399:	learn: 0.2648451	total: 7m 42s	remaining: 11m 33s
400:	learn: 0.2648067	total: 7m 43s	remaining: 11m 32s
401:	learn: 0.2647574	total: 7m 44s	remaining: 11m 30s
402:	learn: 0.2646935	total: 7m 45s	remaining: 11m 29s
403:	learn: 0.2646353	total: 7m 46s	remaining: 11m 28s
404:	learn: 0.2646080	total: 7m 47s	remaining: 11m 27s
405:	learn: 0.2645636	total: 7m 49s	remaining: 11m 26s
406:	learn: 0.2645007	total: 7m 50s	remaining: 11m 25s
407:	learn: 0.2644479	total: 7m 51s	remaining: 11m 24s
408:	learn: 0.2643919	total: 7m 52s	remaining: 11m 22s
409:	learn

542:	learn: 0.2576204	total: 10m 29s	remaining: 8m 49s
543:	learn: 0.2575823	total: 10m 30s	remaining: 8m 48s
544:	learn: 0.2575399	total: 10m 31s	remaining: 8m 47s
545:	learn: 0.2575030	total: 10m 32s	remaining: 8m 46s
546:	learn: 0.2574650	total: 10m 34s	remaining: 8m 45s
547:	learn: 0.2574240	total: 10m 35s	remaining: 8m 43s
548:	learn: 0.2573780	total: 10m 36s	remaining: 8m 42s
549:	learn: 0.2573242	total: 10m 37s	remaining: 8m 41s
550:	learn: 0.2572814	total: 10m 38s	remaining: 8m 40s
551:	learn: 0.2572256	total: 10m 39s	remaining: 8m 39s
552:	learn: 0.2571778	total: 10m 41s	remaining: 8m 38s
553:	learn: 0.2571304	total: 10m 42s	remaining: 8m 37s
554:	learn: 0.2570861	total: 10m 43s	remaining: 8m 35s
555:	learn: 0.2570375	total: 10m 44s	remaining: 8m 34s
556:	learn: 0.2569932	total: 10m 45s	remaining: 8m 33s
557:	learn: 0.2569537	total: 10m 47s	remaining: 8m 32s
558:	learn: 0.2569097	total: 10m 48s	remaining: 8m 31s
559:	learn: 0.2568557	total: 10m 49s	remaining: 8m 30s
560:	learn

693:	learn: 0.2511808	total: 13m 26s	remaining: 5m 55s
694:	learn: 0.2511246	total: 13m 27s	remaining: 5m 54s
695:	learn: 0.2510959	total: 13m 28s	remaining: 5m 53s
696:	learn: 0.2510621	total: 13m 29s	remaining: 5m 51s
697:	learn: 0.2510224	total: 13m 30s	remaining: 5m 50s
698:	learn: 0.2509904	total: 13m 31s	remaining: 5m 49s
699:	learn: 0.2509388	total: 13m 33s	remaining: 5m 48s
700:	learn: 0.2508902	total: 13m 34s	remaining: 5m 47s
701:	learn: 0.2508377	total: 13m 35s	remaining: 5m 46s
702:	learn: 0.2507958	total: 13m 36s	remaining: 5m 45s
703:	learn: 0.2507508	total: 13m 37s	remaining: 5m 43s
704:	learn: 0.2507027	total: 13m 39s	remaining: 5m 42s
705:	learn: 0.2506653	total: 13m 40s	remaining: 5m 41s
706:	learn: 0.2506312	total: 13m 41s	remaining: 5m 40s
707:	learn: 0.2505694	total: 13m 42s	remaining: 5m 39s
708:	learn: 0.2505304	total: 13m 43s	remaining: 5m 38s
709:	learn: 0.2504974	total: 13m 44s	remaining: 5m 36s
710:	learn: 0.2504536	total: 13m 45s	remaining: 5m 35s
711:	learn

844:	learn: 0.2455380	total: 16m 24s	remaining: 3m
845:	learn: 0.2455058	total: 16m 25s	remaining: 2m 59s
846:	learn: 0.2454838	total: 16m 26s	remaining: 2m 58s
847:	learn: 0.2454485	total: 16m 27s	remaining: 2m 57s
848:	learn: 0.2454210	total: 16m 28s	remaining: 2m 55s
849:	learn: 0.2453885	total: 16m 29s	remaining: 2m 54s
850:	learn: 0.2453445	total: 16m 31s	remaining: 2m 53s
851:	learn: 0.2452991	total: 16m 32s	remaining: 2m 52s
852:	learn: 0.2452654	total: 16m 33s	remaining: 2m 51s
853:	learn: 0.2452405	total: 16m 34s	remaining: 2m 50s
854:	learn: 0.2452081	total: 16m 35s	remaining: 2m 48s
855:	learn: 0.2451738	total: 16m 37s	remaining: 2m 47s
856:	learn: 0.2451445	total: 16m 38s	remaining: 2m 46s
857:	learn: 0.2451128	total: 16m 39s	remaining: 2m 45s
858:	learn: 0.2450824	total: 16m 40s	remaining: 2m 44s
859:	learn: 0.2450606	total: 16m 41s	remaining: 2m 43s
860:	learn: 0.2450296	total: 16m 43s	remaining: 2m 41s
861:	learn: 0.2449952	total: 16m 44s	remaining: 2m 40s
862:	learn: 0.

996:	learn: 0.2406130	total: 19m 23s	remaining: 3.5s
997:	learn: 0.2405754	total: 19m 24s	remaining: 2.33s
998:	learn: 0.2405480	total: 19m 25s	remaining: 1.17s
999:	learn: 0.2405228	total: 19m 26s	remaining: 0us


In [48]:
roc_test

[0.8930235264026893,
 0.8891945137309608,
 0.8881240730145268,
 0.8908283688943219,
 0.8897972136014043]

In [49]:
np.mean(roc_test)

0.8901935391287807

In [50]:
roc_train

[0.8963358850352671,
 0.8973134310135156,
 0.8979886276338936,
 0.8971610910669752,
 0.8972308268687155]

In [51]:
np.mean(roc_train)

0.8972059723236733